In [1]:
import numpy as np
import pandas as pd
import scipy 
import spams
import scanpy
import time, sys, traceback, argparse
import os
import tqdm
import statsmodels.api as sma
import statsmodels.stats as sms
import functools
from tqdm.contrib.concurrent import thread_map

from scipy.io import mmread
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

In [2]:
def regress_covariates(dat, cov_mat):
    '''
    Regress out covariates from expression matrix in place. 
    Two orders of magnitude faster than scanpy.pp.regress_out function.
    
    Parameters
    ----------
    dat: AnnData object
    cov_mat: Cell x covariate Dataframe
    '''
    
    cov_mat = pd.get_dummies(cov_mat, drop_first=True) # Convert factors to dummy variables
    cov_means = cov_mat.values.mean(axis = 0) 
    cov_mat = cov_mat.values - cov_means[np.newaxis, :] # Center covariates
    cov_mat = np.c_[np.ones((cov_mat.shape[0], 1)), cov_mat] # Append intercept
    
    if scipy.sparse.issparse(dat.X):
        dat.X = dat.X.todense()

    lmfit = scipy.linalg.lstsq(cov_mat, dat.X, lapack_driver='gelsy')[0]
    resids = dat.X - cov_mat.dot(lmfit)
    dat.X = resids
    
def fit_skew_norm(t_star, t_nulls, side='both'):
    '''
    Compute p-values by fitting skew normal to null distribution. 
    
    Parameters
    ----------
    t_star: Test statistic
    t_null: Null statistics
    side: Which side to compute pvalues (left, right, or both)
    
    Returns
    ---------
    P-value
    '''
    
    if t_star == 0:
        p = 1
    else:
        fit = scipy.stats.skewnorm.fit(t_nulls)
        if side == 'left':
            p = scipy.stats.skewnorm.cdf(t_star, *fit)
        elif side == 'right':
            p = 1 - scipy.stats.skewnorm.cdf(t_star, *fit)
        elif side == 'both':
            p = scipy.stats.skewnorm.cdf(t_star, *fit)
            p = 2 * np.minimum(p, 1 - p)
        else:
            raise ValueError('Wrong side')
    return p

def scale_effs(B, logmeanexp, downsample_num = 25000, log_exp_baseline = 2):
    '''
    Scale effect sizes to mean expression using LOWESS. 
    
    Parameters
    ----------
    B: Perturbation x gene unscaled effect size matrix
    logmeanexp: Vector of log mean expression values to scale to
    downsample_num: Number of effects used to fit curve
    log_exp_baseline: Mean effect magnitude from this log expression is taken as the value to scale to
    
    Returns
    ---------
    B: Perturbation x gene scaled effect size matrix
    scale_factors: Per-gene scale factors 
    '''
    
    data_frac = min(1, downsample_num / np.prod(B.shape))
    
    if B.shape[1] != len(logmeanexp):
        raise ValueError('Number of genes differs')
    rand_idx = np.c_[np.random.randint(0, B.shape[0], downsample_num), 
                     np.random.randint(0, B.shape[1], downsample_num)]
    to_plot = np.c_[logmeanexp[rand_idx[:,1]], np.log(np.abs(B[rand_idx[:,0],rand_idx[:,1]]))]
    to_plot = to_plot[~np.isinf(to_plot[:,1]),:]
    fit = sma.nonparametric.lowess(to_plot[:,1], to_plot[:,0], return_sorted=False, xvals = logmeanexp)
    baseline = fit[min(i for i,x in enumerate(logmeanexp) if x > log_exp_baseline)]
    scale_factors = np.exp(fit - baseline)
    B = B / scale_factors
    return B, scale_factors

def sec_to_str(t):
    '''Convert seconds to days:hours:minutes:seconds'''
    [d, h, m, s, n] = functools.reduce(lambda ll, b: divmod(ll[0], b) + ll[1:], [(t, 1), 60, 60, 24])
    f = ''
    if d > 0:
        f += '{D}d:'.format(D=d)
    if h > 0:
        f += '{H}h:'.format(H=h)
    if m > 0:
        f += '{M}m:'.format(M=m)

    f += '{S}s'.format(S=s)
    return f

def signif(X, n):
    '''Round elements of a pandas DF X to n significant figures'''
    def func(x):
        if x == 0:
            return 0
        else:
            return round(x, n - 1 - int(np.floor(np.log10(abs(x)))))
    return X.applymap(func) 


In [30]:
rank = 40
lambda1 = 0.1
lambda2 = 10

In [31]:
## read in data
cts = mmread('CD14_counts.mtx')
cts = cts.T.todense()
cts.colnames = open("CD14_c_rownames.txt", "r").read().split('\n')
cts.rownames = open("CD14_c_colnames.txt", "r").read().split('\n')

metadata = pd.read_csv('CD14_metadata.csv')
#metadata = metadata.rename(columns = {'Unnamed: 0':'Cell'})
cts.metadata = metadata

P = mmread('CD14_perturbations.mtx')
P = P.todense()
P.rownames = open("CD14_p_rownames.txt", "r").read().split('\n')
#P.rownames = [x.replace('\n', '') for x in P.rownames]
P.colnames = open("CD14_p_colnames.txt", "r").read().split('\n')

if not cts.rownames == P.colnames:
    print('Cell names in perturbation matrix do not match cell names in expression matrix')

In [32]:
#Y = cts
Y = normalize(cts, axis=1, norm='l1')*10000
Y = np.log(Y+1)
Y = Y - Y.mean(axis = 0)

C:\Users\diksh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [33]:
Y[0:5,0:5]

array([[-0.64336837, -0.38034448, -0.01955386,  0.47358266, -0.17888556],
       [-0.64336837, -0.38034448, -0.01955386,  0.46512168,  0.78621616],
       [ 4.29748008, -0.38034448, -0.01955386,  0.26728455,  0.30216438],
       [-0.64336837, -0.38034448, -0.01955386, -4.67356391,  0.0377176 ],
       [-0.64336837, -0.38034448, -0.01955386,  1.08993397, -4.63868408]])

In [34]:
n_guides = P.sum(axis = 0)
nt_names = ['non-targeting', 'safe-targeting']
#ctrl_idx = np.logical_and(n_guides == 1, P.rownames[nt_names].sum(axis = 0).values != 0)
ctrl_idx = np.where(np.isin(metadata['guide.names'],['non-targeting','safe-targeting']))[0].tolist() 
ctrl_exp = Y[ctrl_idx,:].mean(axis = 0)
Y = Y - ctrl_exp
#Y = Y[:,np.array(Y.sum(axis = 0))[0,:] != 0]
Y = Y[:,Y.sum(axis = 0) != 0]
keep_cells = (P.sum(axis = 0) > 0).tolist()[0]
p_mat = np.asfortranarray(P[:,keep_cells].T).astype(float)

In [35]:
W = spams.trainDL(np.asfortranarray(Y.T), K=rank, lambda1=lambda1, iter=50, verbose=False)

In [36]:
W[0:5,0:5]

array([[-0.03160682,  0.00152236, -0.01066969,  0.03307026,  0.0103731 ],
       [-0.0019396 ,  0.00648054, -0.01644538,  0.00588551,  0.00523663],
       [ 0.0010546 , -0.0017476 ,  0.00061856,  0.00116953,  0.00081482],
       [-0.0727696 , -0.0054647 , -0.04238932,  0.01186655, -0.05139222],
       [ 0.03270725,  0.05276391, -0.20046013,  0.00944554,  0.11477929]])

In [37]:
U_tilde = spams.lasso(np.asfortranarray(Y.T), D=W, lambda1=lambda1, verbose=False)
U_tilde = U_tilde[:, keep_cells]
U_tilde = np.asfortranarray(U_tilde.T.todense())
W = W.T

In [38]:
U = spams.lasso(U_tilde, D=p_mat, lambda1=lambda2, verbose=False)
B = U.dot(W)

In [39]:
B.shape

(49, 310)

In [40]:
B[0:5,0:5]

array([[-5.16693041e-02, -3.89618038e-02, -2.14597802e-03,
        -1.96632043e-02, -1.67685822e-01],
       [ 9.25595034e-03,  1.06666017e-02,  6.48506149e-04,
         2.97658420e-03,  2.04052841e-02],
       [ 4.23471698e-02,  1.56852690e-02,  2.99957068e-03,
         8.12775405e-02,  1.41071737e-01],
       [ 1.00989206e-02, -7.27574315e-04,  2.02315123e-06,
         3.58382035e-02,  1.28743913e-01],
       [ 4.67817939e-02,  3.65833848e-02,  6.83720326e-04,
         9.59284163e-02, -9.57275167e-02]])

In [44]:
num_perms = 1000
pvals = np.zeros((B.shape))
for i in range(num_perms):
    p_mat_perm = np.asfortranarray(p_mat[np.random.permutation(p_mat.shape[0]),:])
    U_perm = spams.lasso(U_tilde, D=p_mat_perm, lambda1=lambda2, verbose=False)
    B_perm = U_perm.dot(W)
    temp_indices = B < B_perm
    pvals[temp_indices] = pvals[temp_indices] + 1
pvals /= num_perms
pvals[pvals > 0.5] = 1 - pvals[pvals > 0.5] # get 2-sided pvalues
pvals *= 2 
pvals = (pvals * num_perms + 1) / (num_perms + 1)

In [45]:
pvals.shape

(49, 310)

In [46]:
pvals

array([[0.01098901, 0.002997  , 0.06493506, ..., 0.01098901, 0.04495504,
        0.002997  ],
       [0.64235764, 0.41658342, 0.7002997 , ..., 0.35464535, 0.3966034 ,
        0.06893107],
       [0.05094905, 0.24075924, 0.02897103, ..., 0.18881119, 0.8961039 ,
        0.23476523],
       ...,
       [0.12887113, 0.0969031 , 0.16883117, ..., 0.15484515, 0.20879121,
        0.02697303],
       [0.92607393, 0.62437562, 0.99000999, ..., 0.76223776, 0.88811189,
        0.57442557],
       [0.39060939, 0.81018981, 0.25274725, ..., 0.996004  , 0.73226773,
        0.5964036 ]])

In [48]:
np.savetxt("pvals.csv", pvals, delimiter=",")
np.savetxt("B.csv", B, delimiter=",")